In [2]:
import math
import os
import pandas as pd
import numpy as np
import sklearn.tree

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats

from googletrans import Translator
import copy
import subprocess

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

# Finding Significant Experiments For Products
- __Now I will be separating each product out by its platform, as well as by the NPS question__
    - If a product contains BOTH a question about O365 Suite (i.e."How likely are you to recommend O365 to a friend?") AND tha specific product (i.e. "How likely are you to recommend Word to a friend?") then that product will be separated out as such
    - If the product does not have both questions then it will be left with one

In [4]:
#in order to separate products by NPS question, I need to re-import the data adding in the actual questions
#set proper working directors
os.chdir('C:\\Users\\paperspace\\Desktop\\Microsoft\\AB Testing')

#import data (only first n00k rows as a sample)
chunksize = 100000
df = pd.read_csv('end_user_nps_cy2020_only_e-exp_partial.tsv', delimiter = '\t', chunksize = chunksize)

#read in dataset that contains column names
colnames = pd.read_excel('sample_AB.xlsx', nrows = 1)

df.columns = colnames.columns

#read in df by chunks
chunks = []
for idx, chunk in enumerate(df):
    chunk.columns = colnames.columns
    chunk = chunk[['XpId', 'XpGroupType', 'XpUrl', 'Id','SurveyId', 'SurveyRatingQuestion', 'SurveyType', 'SurveyRating', 'SurveyStartDate', 
                   'ProcessSessionId', 'Platform', 'Product', 'OcvLanguage', 'ClientSubmitTime']]
    chunks.append(chunk)
    print(f'finished importing chunk {idx + 1}')

#concatenate chunks in list, have to pass sort=False to ignore the FutureWarning produced by this
df = pd.concat(chunks, sort = False)

C:\Users\paperspace\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (8,30,170,181,205) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


finished importing chunk 1
finished importing chunk 2
finished importing chunk 3


C:\Users\paperspace\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (30,170,181) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


finished importing chunk 4
finished importing chunk 5


C:\Users\paperspace\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (170,181) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


...

finished importing chunk 58


C:\Users\paperspace\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (8,170,181,205,254) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


finished importing chunk 59
finished importing chunk 60
finished importing chunk 61
finished importing chunk 62
finished importing chunk 63


C:\Users\paperspace\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (8,205) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


finished importing chunk 64


In [32]:
df.to_csv('Logit_Regression_df_with_separate_NPS_questions.csv', index = False)

os.chdir('C:\\Users\\paperspace\\Desktop\\Microsoft\\AB Testing')
df = pd.read_csv('Logit_Regression_df_with_separate_NPS_questions.csv')

#since these questions are all in different languages, and some of the questions are worded differently, I am going to simply filter them by "Office 365" and the name of the
#individual product -- even when this question is in different languages, the name must be in there still hopefully
questions = pd.DataFrame(df['SurveyRatingQuestion'].value_counts()).reset_index().rename(columns = {'index':'Question'})

questions.to_csv('foreign_questions.csv', index = False)

## Different Languages
- __So... there are several different languages that NPS questions were asked in for these surveys. I am going to translate these to English using Google Translate, then merge them back into the whole data set so that I can sort out specific app vs 0ffice 365 Suite questions__

In [4]:
#using Google Translate to translate foreign language questions into english (this may not work on some VPNs, so if you are having trouble with this
# then I recommend you change VPNs)
def translate_text(text, dest_language="en"):
        # Used to translate using the googletrans library
    import json
    translator = Translator()
    try:
        translation = translator.translate(text=text, dest=dest_language)
    except json.decoder.JSONDecodeError:
        # api call restriction
        process = subprocess.Popen(["nordvpn", "d"], stdout=subprocess.PIPE)
        process.wait()
        process = subprocess.Popen(["nordvpn", "c", "canada"], stdout=subprocess.PIPE)
        process.wait()
        return Process_Data.translate_text(text=text, dest_language=dest_language)
    return translation

#translate all questions
question_list = [x for x in questions.Question]

translated_questions = [translate_text(x).text for x in question_list]

questions.insert(loc = 1, column = 'TranslatedQuestions', value = [question for question in translated_questions])

questions.to_csv('translated_quwstions_from_NPS_survey.csv', index = False)

os.chdir('C:\\Users\\paperspace\\Desktop\\Microsoft\\AB Testing\\Logistic Regression')
translated_questions = pd.read_csv('translated_questions_from_NPS_survey.csv').drop('SurveyRatingQuestion', axis = 1) \
                                            .rename(columns = {'TranslatedQuestions':'TranslatedQuestion'})

In [106]:
translated_questions.tail()

,Question,TranslatedQuestion
561,"Mennyire valószínű, hogy ajánlja a Outlook-öt másoknak, ha a véleményét kérnék?",How likely are you to recommend your Outlook or five others when I need a review?
562,À quelle fréquence recommanderiez-vous Outlook à d’autres personnes si vous y êtes invité ?,How would you recommend Outlook frequency to others if prompted?
563,¿Qué probabilidades hay de que recomiendes Outlook a otras personas si te preguntan?,What are the odds of you refer Outlook to others if they ask?
564,¿Qué probabilidades hay de que recomiende Outlook a otras personas si le preguntaran?,What are the odds Outlook to recommend to others if asked?
565,"Hur troligt är det att du skulle rekommendera Outlook för andra, om du fick frågan?","How likely is it that you would recommend Outlook for others, if you were asked?"


In [9]:
#merging translated questions into original df, dropping redundant question column 
df = translated_questions.join(df.set_index(df.SurveyRatingQuestion), on = 'Question').drop(['Question'], axis = 1)

#searching for keywords in the df and separating products out by keyword -- this generates some duplicates becuase some questions include BOTH the
# name of an app (i.e. Word) AND the word "app" as well, so we need to filter out duplicates with boolean logic after concatenating all of them, then filter 
# -- Therea are about 20 surveys that we will not capture filtering out in this manner that refer to "games" or other things that were simply lost in 
#translation and unable to be tied back to a specific product
products = ['365|Office|office', 
            "application|apli|App|app", 
            'Word|word', 'Excel|excel', 
            'Powerpoint|powerpoint|PowerPoint', 'Outlook|outlook', 'Access|access', 'Visi|visi', 
            'OneNote', 'program', 'Proj|proj']

d = {}
for name in products:
    d[name] = pd.DataFrame(df[(df['TranslatedQuestion'].astype(str).str.contains(f'{name}')) | (df['SurveyRatingQuestion'].astype(str).str.contains(f'{name}'))])

#concatenating dfs together to see exactly how many survey responses we didn't capture using the above filtering
dfs = []
for x in d.keys():
    prod = pd.DataFrame(d[x])
    dfs.append(prod)

    
nps_df = pd.concat(dfs, axis = 0).drop_duplicates()

#filter out cases that have NPS questions pertaining to the suite and speicific apps
suite = nps_df[nps_df['TranslatedQuestion'].str.contains('365|Office|office|offi|ofi')]

app = nps_df[~nps_df.index.isin(suite.index)]

#add in a tag 
suite.insert(loc = 0, column = 'NPS Tag', value = 'O365 Suite')
app.insert(loc = 0, column = 'NPS Tag', value = 'App')

In [17]:
#looks like we are in fact only missing 19 columns
nps_df.shape[0], df.shape[0]

(6303374, 6303393)

In [104]:
#here is an example of a data point that we are not interested in, there is no way to tell what this survey is on with respect to an office product, so we drop 
# survey responses that are ambiguous like this, again, this is only 19 surveys out of 6.3 million
df[df['TranslatedQuestion'].str.contains('game')].head(1)

,TranslatedQuestion,XpId,XpGroupType,XpUrl,Id,SurveyId,SurveyRatingQuestion,SurveyType,SurveyRating,SurveyStartDate,ProcessSessionId,Platform,Product,OcvLanguage
443,How likely are you to recommend this game to a friend or colleague?,26895,C,https://ecs.skype.com/?page=ExperimentPage&id=26895,flnps_v2_74d1be15b59a5601a2214a45e313c698,265d54ad-beff-47a1-85c4-2255f0bfee1e,Колика је вероватноћа да ћете ову апликацију препоручити пријатељу или колеги?,NpsStatic,5.0,2/1/2018 12:00:00 AM,517e63f0-5a39-43ed-8f08-96d8e100174e,Windows Desktop,Desktop Word,English


In [26]:
df.to_csv('NPS_data_separated_by_suite_and_app.csv', index = False)

## Start Here
- After cleaning the dfs with the above steps, I exported a cleaned csv and imported it here, so if you are running this code from scratch, start here

In [3]:
#change working directory and import data that we cleaned with the above steps
os.chdir('C:\\Users\\paperspace\\Desktop\\Microsoft\\AB Testing\\Logistic Regression')
df = pd.read_csv('NPS_data_separated_by_suite_and_app.csv')

#separate dfs into app and suite
suite = df[df['NPS Tag'] == 'O365 Suite']
app = df[df['NPS Tag'] == 'App']

In [4]:
df.shape

(6303374, 11)

In [5]:
suite.shape

(2598408, 11)

In [6]:
app.shape

(3704966, 11)

In [30]:
def find_significant_experiments_with_logistic_regression(df1, agg_df, product, solver = 'lbfgs'):
        

    '''
    
    This function takes a df with the above structure and columns. It takes this df, subsets it for a given product (exce, word, access, visio, outlook, powerpoint),    runs the data through a Random Forest Classifier and performs feature selection with the Random Forest's feature_importances_, then determines if there are any 
    linearly dependent columns (with a correlation of 1), drops those redundant features (where one experiment may be another one renamed perhaps), and finally runs
    a logistic regression on the data to determine which experiments are significantly impacting net promoter score.

    Arguments: first argument is a df that must have the structure in the above "head" call, second argument is a string for the name of a product (i.e. 'Word'), 
    last argument is the solver for the logistic regression which is set to Limited Memory Broyden-Fletcher-Goldfarb, the default solver in sklearn
    
    Output -> The output of this function is a tuple containing 1. A preliminary summery of the mode, 2. A second summary of the model (used to take columns from),
    and 3. An output dataframe containing the significant experiment coefficients from the dataset. As such, use of this function call should be as follows:
    
    summary1, summary2, final_output_df = find_significant_experiments_with_logistic_regression(df, 'Word') 
    
    *Comment lines out in first section of this code depending if you want to run analysis on Word, Powerpoint, and Excel combined or not

    '''
    
    print(f'---->---->Running Analysis for {product}<----<----')
    print(' ')
    
    #comment this line out when running analyses if you want to see only unique experiments for each product
    df1 = df1[df1['Product'].astype(str).str.contains(f'{product}')]
    
    #isnert a column with all 1s for one hot encoding of experiments for each individual (the 1 will mark experiments that someone was in)
    df1.insert(loc = 0, column = 'n', value = 1)
    
    #create df for logistic regression, a pivot table that has columns that are experiment IDs, df denotes whether each individual was in the experiment
    log_df = df1.pivot_table(index=['Id'], columns = ['XpId'], values = 'n', fill_value = 0)
    
    #sort values of excel_df by user ID, this is to add in NPS scores for each person as a column to the log_df
    df1 = df1.sort_values('Id').drop_duplicates(subset = 'Id')
    
    #add NPS score to the log_df
    log_df['NPS'] = [x for x in df1['SurveyRating']]
    
    #map 0 (to denote detractors) and 1 (to denote passives and promoters) onto NPS column
    log_df['NPS'] = log_df['NPS'].map({1:0, 2:0, 3:0, 4:1, 5:1})
    
    #create train/test split to feed into Random Forest, we are using this algorithm to filter out important experiments for NPS
    X_train, X_test, y_train, y_test = train_test_split(log_df.drop('NPS', axis = 1), log_df['NPS'], test_size=0.2)
    
    print('Beginning Random Forest Feature Selection')

#     # run gridsearch for best Random Forest params, in this case I am using RandomizedSearchCV rather than GridsearchCV, which has been shown to have similar
#     # performance to Gridsearch, but takes significantly less time 
#     rf = RandomForestClassifier(random_state=0)
#     param_grid = {'n_estimators': [50, 100, 250, 500, 750, 1000, 1250],#'max_features': ['auto', 'sqrt', 'log2'],
#                   'max_depth': [5,10,25,None]}
#     #'oob_score': [True,False]}

#     CV_rfc = RandomizedSearchCV(estimator=rf, n_jobs=-1, param_distributions=param_grid, verbose=10, scoring='neg_log_loss',cv= 5)
#     CV_rfc.fit(X_train, y_train)
    
#     n_estimators, max_depth = CV_rfc.best_params_.values()


    rfc = RandomForestClassifier(random_state = 0, n_estimators = 100, max_depth = 10, oob_score = True)
    rfc.fit(X_train,y_train)
    print(' ')
    print('Scoring Random Forest on Test Data')
    print(rfc.score(X_test,y_test))
    
    #getting feature importances greatest to least
    learners = rfc.feature_importances_.argsort()[::-1]
    
    features = pd.DataFrame(X_train.columns[learners], rfc.feature_importances_[learners])
    
    #only using features that had more than .015 splits
    features = features[features.index>0.015]
    
    #putting the best features into a list to use later to filter variables out of log_df
    tree_best = features.XpId.values.tolist()
    
    #creating data frame containing features identified as important by Random Forest
    X = log_df[[x for x in tree_best]]
    
    #insert a constant for logistic Regression
    X.insert(loc = 0, column = 'constant', value = 1)
    
    #ranking columns in the dataset, there should be no tied rankings for regression to run properly
    print(' ')
    print('first value of each line should always be second value minus 1')
    print(' ')
    rank_test = X
    for i in range(rank_test.shape[1]):
        df_to_rank = rank_test.iloc[:,:i+1]
        print(i,np.linalg.matrix_rank(df_to_rank))
    print(' ')
    
    #redundant columns check
    matrix = X.corr()==1
    linear_dependents = []
    for col in matrix.columns:
        #print(col)
        linear_dependents.append(list(matrix[col].index[matrix[col]==True]))
    linear_dependents = [x for x in linear_dependents if len(x) > 1]
    
    #if there are linear dependents, then store them in a list and print them out in the report
    if not linear_dependents: #evaluates whether the list contains info -- if not then print there is nothing there, same for next loop
        print('No Linear Dependents Present')
    else:
        print('Here is a List of Linear Dependents:')
        redundant_cols = [x for x in linear_dependents if len(x)>1]
        print(redundant_cols)
    
    #if there are dependents then store them in a list, if not then just print a space
    if not linear_dependents:
        print(' ')
    else:
        dependents = []
        for x in linear_dependents:
            if len(x) > 1:
                dependents.append(x)


        #create a list of linearly dependent features, we are going to drop these
        to_drop = pd.DataFrame(dependents).drop_duplicates().iloc[:,0].tolist()

        #accounting for more than 2 linearly dependent columns -- should probably add in functionality for more than 3 in the future, but I have only seens 3 so far
        a = pd.DataFrame(dependents).drop_duplicates()
        b = a.fillna('missing')
        second_redundant_column_out_of_3_linear_dependents = []
        for idx, row in b.iterrows():
            if (len(row) == 3) & ('missing' not in list(row)):
                second_redundant_column_out_of_3_linear_dependents.append(row[1])

        #extend the second out of 3 linear dependents to the "to_drop" list
        to_drop.extend([col for col in second_redundant_column_out_of_3_linear_dependents])
    
        #drop linearly dependent features
        X.drop(to_drop, axis = 1, inplace = True)  
    
        print(' ')
        print('first value of each line should always be second value minus 1 (After Dropping Perfectly Correlated Features)')
        print(' ')
        
        #perform rank test again to make sure that columns are not duplicates
        rank_test = X
        for i in range(rank_test.shape[1]):
            df_to_rank = rank_test.iloc[:,:i+1]
            print(i,np.linalg.matrix_rank(df_to_rank))
                  
    
    print(' ')
    print(f'Running Logistic Regression for {product}')
    
    #if we set a solver then use it
    if solver != None:
        
        #instantiate Logistic Regression, print out summary
        logit = sm.Logit(log_df['NPS'], X)

        flogit = logit.fit(method = solver, maxiter = 2000) #setting max_iter high so models have a better chance of converging

        summary1 = flogit.summary()

        summary2 = flogit.summary2()
    else:
        logit = sm.Logit(log_df['NPS'], X)
        
        flogit = logit.fit(maxiter = 2000)
        
        summary1 = flogit.summary()
        
        summary2 = flogit.summary2()
        
    #I created this function to format the final table for each output, that way I only have to concatenate them at the very end
    def make_final_table(summary2):
        '''
        
        This function creates a final output table with the odds ratio, upper and lower bounds of the ratio, final probability, and lift of the experiment

        Arguments: summary2 produced by the logit in the previous step

        '''

        #get the summary table
        coefficients = summary2.tables[1].round(2)
        
        #create column for odds ratio
        coefficients['Odds Ratio'] = np.exp(coefficients['Coef.'])
        
        #create column for the lower bound of the odds ratio
        coefficients['O.R.LB'] = np.exp(pd.DataFrame(summary2.tables[1]).iloc[:, -2])
        
        #create column for thet upper bound of the odds ratio
        coefficients['O.R.UP'] = np.exp(pd.DataFrame(summary2.tables[1]).iloc[:, -1])
        
        #grab cases that have a probability of less than.1 from the table (same threshold used in Jared's analysis)
        coefficients = coefficients[coefficients['P>|z|'] < .1]
        
        #create column for the final probability
        coefficients['FinalProbability'] = np.exp(coefficients['Coef.'].round(1)) * log_df['NPS'].value_counts(normalize = True).loc[1]
        coefficients['FinalProbability'] = coefficients['FinalProbability'].mask(coefficients['FinalProbability'] >= 1, 0.99)
        
        #create column for the lift 
        coefficients['Lift'] = coefficients['FinalProbability'] - log_df['NPS'].value_counts(normalize = True).loc[1]

        #return coefficient df
        return coefficients

    coefficients = make_final_table(summary2)
        
    summary = flogit.summary()
    
    def create_final_table(df2, df3, product):
        
        '''
        This function creates the final formatted table, merging in treatment group sizes, product name, and treatment share out of the total flight
        
        '''
        
        groups = df2.groupby(['XpId', 'Product']).XpGroupType.value_counts().unstack().reset_index()

        same_ids = df3[df3.index != 'constant'].reset_index().rename(columns = {'index':'XpId'}).sort_values('XpId')

        groups = groups[(groups['XpId'].isin(same_ids.XpId)) & (groups['Product'].str.contains(f'{product}'))].sort_values('XpId')

        final_output = same_ids.merge(groups).fillna(0)

        final_output['TreatmentCount'] = [x for x in final_output['T']]
        final_output['FlightCount'] = final_output['TreatmentCount'] + final_output['C']
        final_output.drop(['T', 'C'], axis = 1, inplace = True)
        final_output['TreatmentShre'] = final_output['TreatmentCount'] / final_output['FlightCount']

        final_output = final_output[(final_output['TreatmentCount'] > 30) & (final_output['FlightCount'] > 60)]
    
        return final_output
    
    final_output = create_final_table(agg_df, coefficients.round(2), product)
    
    print(' ')
    print('---->---->Function Successfully Run<----<----')
    print(' ')
        
    return summary, final_output

# Run Experiment for EVERYTHING
- This will take a long time, but it will end with all final dfs created

In [31]:
%%time


#create list oc products to iterate through
products = ['Word', 'Excel', 'Powerpoint', 'Outlook', 'OneNote', 'Access', 'Visio']

#store all results in lists, concatenate them at the end
both_final_outputs = []
both_summaries = []
suite_final_outputs = []
suite_summaries = []
app_final_outputs = []
app_summaries = []

#run experiments for suite and app
for product in products:
    try:
        summary, final_output = find_significant_experiments_with_logistic_regression(df, df, product, 'lbfgs')
        both_final_outputs.append(final_output)
        both_summaries.append(summary)
    except:
        pass
        print(f'Not Enough Data to Use for {product}')

#run experiments for suite        
for product in products:
    try:
        summary, final_output = find_significant_experiments_with_logistic_regression(suite, suite, product, 'lbfgs')
        suite_final_outputs.append(final_output)
        suite_summaries.append(summary)
    except:
        pass
        print(f'Not Enough Data to Use for {product}')
#run experiments for app
for product in products:
    try:
        summary, final_output = find_significant_experiments_with_logistic_regression(app, app, product, 'lbfgs')
        app_final_outputs.append(final_output)
        app_summaries.append(summary)
    except:
        pass
        print(f'Not Enough Data to Use for {product}')
        
        
#concatenate everything
both_final = pd.concat(both_final_outputs, axis = 0)
app_final = pd.concat(app_final_outputs, axis = 0)
suite_final = pd.concat(suite_final_outputs, axis = 0)

---->---->Running Analysis for Word<----<----
 
Beginning Random Forest Feature Selection
 
Scoring Random Forest on Test Data
0.8085949764521193
 
first value of each line should always be second value minus 1
 
0 1
1 2
2 3
3 4
4 5
5 6
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
18 17
19 18
20 19
21 20
22 21
23 22
 
Here is a List of Linear Dependents:
[[33079, 33082], [33079, 33082], [28677, 29662], [28677, 29662]]
 
first value of each line should always be second value minus 1 (After Dropping Perfectly Correlated Features)
 
0 1
1 2
2 3
3 4
4 5
5 6
6 7


C:\Users\paperspace\Anaconda3\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


7 8
8 9
9 10
10 11
11 12
12 13
13 14
14 15
15 16
16 17
17 18
18 19
19 20
20 21
21 22
 
Running Logistic Regression for Word
 
---->---->Function Successfully Run<----<----
 
---->---->Running Analysis for Excel<----<----
 
Beginning Random Forest Feature Selection
 
Scoring Random Forest on Test Data
0.8222112606581636
 
first value of each line should always be second value minus 1
 
0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14
14 15
15 16
16 17
17 18
18 19
19 20
20 21
 
No Linear Dependents Present
 
 
Running Logistic Regression for Excel
 
---->---->Function Successfully Run<----<----
 
---->---->Running Analysis for Powerpoint<----<----
 
Beginning Random Forest Feature Selection
 
Scoring Random Forest on Test Data
0.8210852517075639
 
first value of each line should always be second value minus 1
 
0 1
1 2
2 3
3 4
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
18 18
19 19
20 20
21 21
 
Here is a List of Linear Dependents:
[[33082, 330

C:\Users\paperspace\Anaconda3\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


 10
10 11
11 12
12 13
13 14
14 15
15 16
16 17
17 18
18 19
19 20
20 21
 
Running Logistic Regression for Powerpoint
 
---->---->Function Successfully Run<----<----
 
---->---->Running Analysis for Outlook<----<----
 
Beginning Random Forest Feature Selection
 
Scoring Random Forest on Test Data
0.753732790382005
 
first value of each line should always be second value minus 1
 
0 1
1 2
2 3
3 3
4 4
5 5
6 6
7 7
8 8
9 8
10 9
11 10
12 11
13 12
14 12
15 13
16 14
17 15
18 16
19 17
20 18
 
Here is a List of Linear Dependents:
[[29662, 29661, 28677], [29662, 29661, 28677], [29662, 29661, 28677], [33082, 33079], [33082, 33079]]
 
first value of each line should always be second value minus 1 (After Dropping Perfectly Correlated Features)
 
0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14


...

 
first value of each line should always be second value minus 1 (After Dropping Perfectly Correlated Features)
 
0 1
1 2
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
 
Running Logistic Regression for Outlook
 
---->---->Function Successfully Run<----<----
 
---->---->Running Analysis for OneNote<----<----
 
Beginning Random Forest Feature Selection
 
Scoring Random Forest on Test Data
0.7817248666599578
 
first value of each line should always be second value minus 1
 
0 1
1 2
2 3
3 4
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 24
26 25
 
Here is a List of Linear Dependents:
[[38236, 43163], [38236, 43163], [28677, 29662], [28677, 29662]]
 
first value of each line should always be second value minus 1 (After Dropping Perfectly Correlated Features)
 
0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14
14 15
15 16
16 17
17

C:\Users\paperspace\Anaconda3\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


 18
18 19
19 20
20 21
21 22
22 23
23 24
24 25
 
Running Logistic Regression for OneNote
 
---->---->Function Successfully Run<----<----
 
---->---->Running Analysis for Access<----<----
 
Beginning Random Forest Feature Selection
 
Scoring Random Forest on Test Data
0.7050359712230215
 
first value of each line should always be second value minus 1
 
0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
 
No Linear Dependents Present
 
 
Running Logistic Regression for Access
 
---->---->Function Successfully Run<----<----
 
---->---->Running Analysis for Visio<----<----
 
Beginning Random Forest Feature Selection
 
Scoring Random Forest on Test Data
0.7929292929292929
 
first value of each line should always be second value minus 1
 
0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14
14 15
15 15
 
Here is a List of Linear Dependents:
[[38596, 38714], [38596, 38714]]
 
first value of each line should always be second value minus 1 (After Dropping Perfectly Correlated

C:\Users\paperspace\Anaconda3\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


 
---->---->Function Successfully Run<----<----
 
Wall time: 8min 28s
